## Cold Email Generator

In [3]:
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#Creating model
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature="0.7",
    max_retries=2,
    max_tokens=2000
)
# llm.invoke("Hello")

In [7]:
#Steps 
#Create a webbaseloader form document loader
#Which, will convert data into document
#Pass it to llm to generate important json fileds for further processing
#Store that JSON into a vectore store


In [122]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://careers.ibm.com/job/20978143/software-developer-intern-2025-san-jose-ca/?codes=WEB_Search_INDIA")

page_data= loader.load().pop().page_content
page_data

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSoftware Developer Intern: 2025 - US | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\nSkip to content\n\n\n\nToggle NavigationIBM CareersEnglish简体中文FrançaisFrançais Canadien日本語PortuguêsEspañolDeutschIBM Careers IBM Careers      English    简体中文    Français    Français Canadien    日本語    Português    Español    Deutsch      \n \n\n\n\n\n\nJob Details2024-01-04T09:25:13+00:00 \n\n724766BR\n\n\nSoftware Developer Intern: 2025    Multiple Locations See All  San Jose  Lowell  Poughkeepsie  Austin    Software Engineering Internship   Apply now Software Developer Intern: 2025    Multiple Locations See All  San Jose  Lowell  Poughkeepsie  Austin    Software Engineering Internship   Apply now IntroductionAt IBM, work is more than a job – it’s a calling: To build. To design. To code. To consult. To think along with clients and sell. To make markets. To invent. To collaborate. Not just to do somet

### Creating a chain for LLM

In [126]:
#Passing docuemnt data to llm to get the important JSON Fileds
from langchain.prompts import PromptTemplate

prompt_template_text = PromptTemplate(
    input_variables=["document","company_name"],
    template="""
        You Are a a Cold Email Generator Assiatant. Who's job is to generate a cold emails based on the documents provided by the user.
        Task : Your task is to analiyze whole document provided to you and generate a JSON Fileds.
        Which Are : job_title,required_skills,preferred_skills,exerience,job_description

        The Above Filed are the must fileds. Which you have to generate. 

        Apart, from this you can generate some additional fields into a json which seems to be important for writing a email for job. But, the specifiled fileds are the must ones.

        ## NO PREAMBLE
        ## ONLY RETURN VALID JSON

        The document is given below,
        {document}

        Do not include heading. Not even anything apart from JSON reponse(nothing else).
    """
)

# The sending company name is  {company_name}. Also include the company_name into json as sending_company.

In [ ]:
from langchain.chains import SequentialChain
from langchain.chains import LLMChain

#1st chain
prompt_text = LLMChain(llm=llm,prompt=prompt_template_text,output_key="json")

#Creating a 2nd chain
prompt_template_email = PromptTemplate(
    input_variables=["json","company_name"],
    template="""I Have give a {json} file to you.
    
        You task is to generate a professional cold email based on the given JSON file.
        

        Which is service based company and would like client to hire them for the job.
        It should be a best combination of professional and marketing email.
        The cold email must be professional email from companay {company_name}.
        Don't include preamble in response.
        ## NO PREAMBLE.
        ## ONLY VALID INFORMATION.
    """
)
# Read from json named as sending_company and write it into a from section of email.
#2nd chain 
prompt_email = LLMChain(llm=llm,prompt=prompt_template_email,output_key="cold_email")

In [ ]:
chain = SequentialChain(
    chains=[prompt_text,prompt_email],
    input_variables=["document","company_name"],
    output_variables=["json","cold_email"],
    verbose=True
)

#Running the chain
response= chain({"document":page_data,"company_name":"DRC Systems India Limited."})



> Entering new SequentialChain chain...

> Finished chain.


In [ ]:
print(response["cold_email"])

### Embeddings

In [127]:
#Getting JSON Data

prompt_text = prompt_template_text.format(document=page_data,company_name="DRC Systems India Limited.")
json_response = llm.invoke(prompt_text)

In [128]:
print(json_response.content)

{
"job_title": "Software Developer Intern: 2025",
"required_skills": [
    "Familiarity with one or more programming languages such as Java, Python, C++, or JavaScript",
    "Experience with software development tools such as Git, IDEs (e.g., Eclipse, IntelliJ), and version control systems",
    "Strong analytical and problem-solving abilities with a keen attention to detail",
    "Excellent verbal and written communication skills, with the ability to collaborate effectively in a team environment",
    "Willingness to learn new technologies and adapt to changing project requirements",
    "Ability to work both independently and as part of a collaborative team"
],
"preferred_skills": [
    "Experience with web development frameworks (e.g., React, Angular) or mobile development (e.g., Android, iOS)",
    "Exposure to cloud platforms such as IBM Cloud, AWS, or Azure",
    "Basic understanding of databases, including SQL and NoSQL databases",
    "Familiarity with Agile development practic

In [134]:
prompt_email = prompt_template_email.format(json=json_response,company_name="DRC Systems India Limited.")

email_response = llm.invoke(prompt_email)
print(email_response.content)

Here is a professional cold email based on the given JSON file:

Subject: Expert Software Development Services for Your Organization

Dear [Hiring Manager's Name],

I hope this email finds you well. My name is [Your Name] and I am reaching out from DRC Systems India Limited, a leading service-based company specializing in software development solutions.

We came across your job posting for a Software Developer Intern and were impressed by your organization's commitment to delivering high-quality software solutions. Our team of expert software developers has extensive experience in developing scalable, efficient, and reliable software applications that meet our clients' needs.

Our team's skills align perfectly with your requirements, including proficiency in programming languages such as Java, Python, C++, and JavaScript, as well as experience with software development tools like Git, IDEs, and version control systems. We also possess strong analytical and problem-solving abilities, ex

In [100]:
#Embedding json response 

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


/home/niteshchauhan/Desktop/tasks/Lang-chain/Projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Chromadb

In [109]:
from langchain_community.vectorstores import Chroma

vectore_store = Chroma(
    collection_name="json_data",
    embedding_function=embeddings,
    persist_directory="./cold_email_db"
)


In [110]:
#Initializing Chromadb
import chromadb

#Getting chromadb from lacal storage
persistant_client= chromadb.PersistentClient()

#Getting collection 
collection = persistant_client.get_or_create_collection("json_data")

#Adding json data to collection
collection.add(
    ids=["json_data"],
    documents=[json_response.content]
)


#Embeding resultant data
vectore_store_from_client = Chroma(
    client=persistant_client,
    collection_name="json_data",
    embedding_function=embeddings
)

Add of existing embedding ID: json_data
Insert of existing embedding ID: json_data


In [ ]:
#Now, the vectore database is created 
#So, searching the database
results = collection.query(
    query_texts=["Skills"],
    n_results=1
)
results

### Using chromadb for email generation.

In [117]:
#Using retriver for getting content of the email.
retriever = vectore_store_from_client.as_retriever()